In [ ]:
import nbformat as nbf

nb = nbf.v4.new_notebook()
cells = []

# Celda 1: Título y objetivos
cells.append(nbf.v4.new_markdown_cell("""# Análisis de Clustering con K-Means: Caso Titanic

Este cuaderno tiene fines educativos y te guiará por el flujo clásico de un análisis de clustering con el algoritmo **K-Means**, usando el famoso set de datos del Titanic.

**Objetivos:**
- Preparar y escalar datos.
- Identificar el número óptimo de clústers usando diferentes métricas (Elbow, Silhouette, Gap).
- Aplicar el algoritmo K-Means.
- Visualizar y analizar los resultados.
"""))

# Celda 2: Teoría básica sobre K-Means
cells.append(nbf.v4.new_markdown_cell("""
## ¿Qué es K-Means?

**K-Means** es un algoritmo de *clustering* no supervisado que tiene como objetivo dividir un conjunto de datos en *k* grupos (clusters) homogéneos según sus características. El procedimiento es:

1. Inicializar aleatoriamente k centroides.
2. Asignar cada punto de datos al centroide más cercano.
3. Recalcular la posición de cada centroide como la media de los puntos asignados a cada uno.
4. Repetir los pasos 2-3 hasta que los centroides ya no cambien significativamente.

**¿Para qué sirve?**  
Para encontrar grupos "naturales" en los datos, identificar segmentos de usuarios, patrones de consumo, etc.

**Limitaciones**:
- Hay que definir k (cantidad de clusters) antes de ejecutar el algoritmo.
- No funciona bien si los clusters tienen formas no esféricas o tamaños muy diferentes.
"""))

# Celda 3: Librerías y carga de datos
cells.append(nbf.v4.new_code_cell("""\
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Cargamos el dataset Titanic desde la web
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

df.head()
"""))

# Celda 4: Limpieza y selección de variables
cells.append(nbf.v4.new_markdown_cell("""
## Preparación y Escalado de los Datos

Seleccionamos solo variables numéricas para el clustering y completamos valores nulos.
"""))
cells.append(nbf.v4.new_code_cell("""\
features = ['Age', 'Fare', 'SibSp', 'Parch']
df_features = df[features].copy()

# Completamos nulos en 'Age' con la media
df_features['Age'] = df_features['Age'].fillna(df_features['Age'].mean())
df_features.isnull().sum()
"""))

# Celda 5: Escalado de datos
cells.append(nbf.v4.new_code_cell("""\
# Escalamos los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_features)

pd.DataFrame(X_scaled, columns=features).head()
"""))

# Celda 6: Selección del número óptimo de clusters
cells.append(nbf.v4.new_markdown_cell("""
## ¿Cuántos clusters elegir?  
Utilizaremos tres métodos: **Elbow (Codo)**, **Silhouette (Silueta)** y (opcional) **Gap Statistic**.
"""))

# Elbow
cells.append(nbf.v4.new_code_cell("""\
inertia = []
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(6,4))
plt.plot(K, inertia, 'o-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del codo (Elbow)')
plt.show()
"""))

# Silhouette
cells.append(nbf.v4.new_code_cell("""\
silhouette_scores = []
K2 = range(2, 11)
for k in K2:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(6,4))
plt.plot(K2, silhouette_scores, 'o-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Score de silueta')
plt.title('Método de la Silueta')
plt.show()
"""))

# Celda teoría gap statistic
cells.append(nbf.v4.new_markdown_cell("""
> El método **Gap Statistic** compara la inercia de los clusters reales con la inercia obtenida en datos simulados aleatorios, ayudando a determinar si los clusters encontrados son significativamente mejores que una partición aleatoria.  
> Por simplicidad, lo omitimos aquí, pero puedes consultarlo si tienes instalado el paquete `gap-stat`.
"""))

# Celda 7: Implementación de K-Means
cells.append(nbf.v4.new_markdown_cell("""
## Aplicando K-Means con el número óptimo de clusters

Según los gráficos anteriores, elige el valor de k más adecuado (por ejemplo, 3 o 4) y entrena el modelo.
"""))
cells.append(nbf.v4.new_code_cell("""\
k_opt = 3  # Cambia según los resultados obtenidos arriba
kmeans = KMeans(n_clusters=k_opt, n_init=10, random_state=42)
labels = kmeans.fit_predict(X_scaled)
df['Cluster'] = labels
df['Cluster'].value_counts()
"""))

# Celda 8: Gráficos y análisis de resultados
cells.append(nbf.v4.new_markdown_cell("""
## Análisis y Visualización de los Clusters

Veamos cómo se distribuyen las variables según cada cluster.
"""))
for feature in ['Age', 'Fare', 'SibSp', 'Parch']:
    cells.append(nbf.v4.new_code_cell(f"""\
plt.figure(figsize=(7,4))
sns.boxplot(x='Cluster', y='{feature}', data=df)
plt.title('Boxplot de {feature} por cluster')
plt.show()
"""))

cells.append(nbf.v4.new_code_cell("""\
# Estadísticas promedio por cluster
df.groupby('Cluster')[['Age', 'Fare', 'SibSp', 'Parch']].mean()
"""))

# Celda 9: Interpretación de resultados
cells.append(nbf.v4.new_markdown_cell("""
### Interpretación de los resultados

- Observa si los clusters se diferencian claramente según edad, tarifa, hermanos o hijos a bordo.
- Preguntas guía:
    - ¿Hay clusters que agrupan pasajeros más jóvenes?
    - ¿Hay grupos que tienden a pagar más tarifa?
    - ¿Algún cluster concentra familias grandes?
- Estos patrones pueden relacionarse con clases sociales, tipo de ticket, o características familiares de los pasajeros.

**Recuerda:** En clustering, la interpretación depende mucho del contexto del negocio y puede requerir analizar más variables o combinar con datos categóricos (por ejemplo, sexo, clase).
"""))

# Celda 10: Visualización PCA (adicional)
cells.append(nbf.v4.new_code_cell("""\
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
plt.figure(figsize=(7,5))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=labels, palette='Set2')
plt.title('Clusters visualizados en el plano PCA')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.show()
"""))

cells.append(nbf.v4.new_markdown_cell(
